**通过对cifar10同时来训练一个生成模型和一个判别模型**

In [8]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [9]:
# define the parameter
batchSize = 256
imageSize = 64
beta1 = 0.5
lr = 0.0002

In [10]:
pwd

'/Users/creekee/Work/FastAi/11.18/石任梁'

In [11]:
# download CIFAR10 train_dataset
train_dataset = dset.CIFAR10(root='dataset', 
                             train=True, 
                             transform=transforms.Compose([
                                 transforms.Scale(64), # This transform is deprecated in favor of Resize.
                                 transforms.CenterCrop(64), # Crops the given PIL Image at the center.
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), 
                               ]),
                            download=True)

In [ ]:
import torch.nn as nn
nn.

In [ ]:
# Combines a dataset and a sampler, and provides single- or multi-process iterators over the dataset.
dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size= batchSize,
                                           shuffle=True)

In [5]:
# create the path to the output files
if not os.path.exists('outpath'):
    os.mkdir('outpath')

In [6]:
manualSeed = int()

if manualSeed is None:
    manualSeed = random.randint(1, 10000)

random.seed(manualSeed)

# Sets the seed for generating random numbers. And returns a torch._C.Generator object.
torch.manual_seed(manualSeed)

In [7]:
# initialize the weights and bias
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [8]:
nv = 100 # noise vector
nc = 3 # number of chanels
netG = '' # the generate network model
netD = '' # the discrimitive network model

In [9]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        self.main = nn.Sequential(
            # input is Z(noise vector), give to a convolution
            nn.ConvTranspose2d( nv, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            # the output of the previous layer as the input
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            # the output of the previous layer as the input
            nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            # the output of the previous layer as the input
            nn.ConvTranspose2d(64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # the output of the previous layer as the input
            nn.ConvTranspose2d(64, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        output = self.main(input)
        
        return output

[ConvTranspose2d](http://pytorch.org/docs/master/nn.html)

In [10]:
# generate network
netG = _netG()
netG.apply(weights_init)

_netG (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU (inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh ()
  )
)

In [11]:
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # the output of the previous layer as the input
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # the output of the previous layer as the input
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # the output of the previous layer as the input
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (64*8) x 4 x 4
            nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)

In [7]:
torch.nn.Conv2d？

SyntaxError: invalid character in identifier (<ipython-input-7-25fd5864f4d6>, line 1)

In [12]:
# the discrimitive network model
netD = _netD()
netD.apply(weights_init)

_netD (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid ()
  )
)

In [13]:
# define the loss function
criterion = nn.BCELoss()

In [14]:
input_data = torch.FloatTensor(batchSize, 3, imageSize, imageSize) # define the shape of the input_data
noise = torch.FloatTensor(batchSize, nv, 1, 1) # define the shape of the noise vector
fixed_noise = torch.FloatTensor(batchSize, nv, 1, 1).normal_(0, 1)
label = torch.FloatTensor(batchSize) # define the label
real_label = 1
fake_label = 0
iteration = 10

In [15]:
fixed_noise = Variable(fixed_noise) 

In [16]:
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [17]:
# train the both of the two models
for epoch in range(iteration):
    for i, data in enumerate(dataloader, 0):
        netD.zero_grad() # Clears the gradients of all optimized Variable s.
        train_value, _ = data # get the train value
        batch_size = train_value.size(0) # get the batch size
        input_data.resize_as_(train_value).copy_(train_value) # resize the input size to fit the train value size
        label.resize_(batch_size).fill_(real_label) # resize the labels
        inputv = Variable(input_data) # the input vector
        labelv = Variable(label) # the output vector
        
        
        output = netD(inputv) # prediction of the discrimitive model on real sample
        errD_real = criterion(output, labelv) # errors of discrimitive model on real sample
        errD_real.backward()
        D_x = output.data.mean()
        
        
        noise.resize_(batch_size, nv, 1, 1).normal_(0, 1) # resize the noise vector
        noisev = Variable(noise)
        fake = netG(noisev) # generating new samples 
        labelv = Variable(label.fill_(fake_label)) 
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv) # errors of discrimitive model on real sample
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()
        
        netG.zero_grad() 
        labelv = Variable(label.fill_(real_label))  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()
        
        if i % 100 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' 
                  % (epoch, iteration, i, len(dataloader), 
                     errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
        
        if i % 100 == 0:
            vutils.save_image(train_value,
                    '%s/real_samples.png' % 'outpath',
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03d.png' % ('outpath', epoch),
                    normalize=True)

[0/10][0/196] Loss_D: 1.6661 Loss_G: 5.3887 D(x): 0.4926 D(G(z)): 0.5081 / 0.0070


KeyboardInterrupt: 

In [ ]:
# save the checkpoint
torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outpath, epoch))
torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outpath, epoch))